In [18]:
import glob
import numpy as np
from keras.preprocessing.image import load_img, img_to_array

# original_image内のファイルのパスををすべて取得する。
files = glob.glob('original_image/*.jpg')

# imgs内に元のデータを入れる
# file path => Python Imaging Library => ndarray
imgs = np.array([img_to_array(load_img(f, target_size=(544, 384))) for f in files])

# imgs内にいくつファイルが入っているか
imgs.shape

(300, 544, 384, 3)

In [23]:
import os

th = 128

lavel_files = glob.glob('lavel_image/*.png')

# labels内にラベルデータを入れる
# file path => Python Imaging Library => ndarray
lavels = np.array([img_to_array(load_img(f, target_size=(544, 384))) for f in lavel_files])

for file in lavels:
    file[file < th] = 0
    file[file >= th] = 1

labels.shape

(300, 544, 384, 3)